In [11]:
'''
Girish Rajani-Bathija
A20503736
CS 577 - F22
Assignment 2 Part 2 Question 1 - Writing a Training Loop from Scratch
'''

from keras.utils.np_utils import to_categorical
from keras import models, layers
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#Load iris dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

def load_iris_data(url):
    dataframe = pd.read_csv(url, header=None)
    X = dataframe.iloc[:, :-1]
    Y = dataframe.iloc[:, -1]
    train_data, test_data, train_labels, test_labels = train_test_split(X, Y, test_size=0.3)
    return train_data, test_data, train_labels, test_labels

train_data, test_data, train_labels, test_labels = load_iris_data(url)

#normalize training and testing data
train_data = preprocessing.normalize(train_data)
test_data = preprocessing.normalize(test_data)

#encode and vectorize labels using label encoder and categorical encoding
lbl_encoder = LabelEncoder()
train_labels_encoded = lbl_encoder.fit_transform(train_labels)
test_labels_encoded = lbl_encoder.fit_transform(test_labels)

one_hot_train_labels = to_categorical(train_labels_encoded)
one_hot_test_labels = to_categorical(test_labels_encoded)


#Build the network
model = models.Sequential()
model.add(layers.Dense(64, activation='sigmoid', input_shape=(4,)))
model.add(layers.Dense(32, activation='sigmoid'))
model.add(layers.Dense(3, activation='softmax'))
#try 4,4,3 for units per layer or 32,16,3

#Compile the model using optimizer and loss function
optimizer = keras.optimizers.Adam(learning_rate=0.01)
loss_fn = keras.losses.CategoricalCrossentropy(from_logits=False)

train_acc_metric = keras.metrics.CategoricalAccuracy()
val_acc_metric = keras.metrics.CategoricalAccuracy()

#Create validation set from training data
x_val = train_data[:25]
partial_x_train = train_data[25:]
y_val = one_hot_train_labels[:25]
partial_y_train = one_hot_train_labels[25:]

batch_size = 5

#Prepare training and validation dataset for training loop
train_dataset = tf.data.Dataset.from_tensor_slices((partial_x_train, partial_y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)


epochs = 20
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    #Iterate through batches in the training data
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):

        with tf.GradientTape() as tape:
            
            #forward pass
            logits = model(x_batch_train, training=True)  

            #Compute the loss value
            loss_value = loss_fn(y_batch_train, logits)

        #Backpropagation to calculate the gradient
        grads = tape.gradient(loss_value, model.trainable_weights)

        #Update weights using gradients derived from backpropagation
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        #Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)
        
    #Print training loss at the end of each epoch
    print("Training loss over epoch: %.4f" % (float(loss_value),))        
            
    # Print training loss at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    #Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()


    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        
        #Update validation metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    
    val_loss_value = loss_fn(y_batch_val, val_logits)
    
    #Print validation loss and accuracy at the end of each epoch
    print("Validation loss over epoch: %.4f" % (float(val_loss_value),))
    print("Validation acc over epoch: %.4f" % (float(val_acc),))


Start of epoch 0
Training loss over epoch: 1.6863
Training acc over epoch: 0.4250
Validation loss over epoch: 0.7996
Validation acc over epoch: 0.2800

Start of epoch 1
Training loss over epoch: 1.0131
Training acc over epoch: 0.3375
Validation loss over epoch: 1.1981
Validation acc over epoch: 0.4000

Start of epoch 2
Training loss over epoch: 1.0209
Training acc over epoch: 0.4125
Validation loss over epoch: 1.0087
Validation acc over epoch: 0.2800

Start of epoch 3
Training loss over epoch: 1.0627
Training acc over epoch: 0.3875
Validation loss over epoch: 1.0447
Validation acc over epoch: 0.3200

Start of epoch 4
Training loss over epoch: 1.0180
Training acc over epoch: 0.4125
Validation loss over epoch: 1.0829
Validation acc over epoch: 0.6800

Start of epoch 5
Training loss over epoch: 1.1494
Training acc over epoch: 0.4875
Validation loss over epoch: 1.0952
Validation acc over epoch: 0.6800

Start of epoch 6
Training loss over epoch: 1.0349
Training acc over epoch: 0.6250
Valid